In [7]:
import numpy as np
import matplotlib.pyplot as plt

# Datos proporcionados
I = np.array([
    0.01596, 0.01586, 0.01576, 0.01554, 0.01495, 0.01459, 0.01423, 0.01407, 0.01328, 0.01299,
    0.01271, 0.01259, 0.01196, 0.01138, 0.01086, 0.01039, 0.00995, 0.00956, 0.00885, 0.00824,
    0.0077, 0.00724, 0.00682, 0.0053, 0.00435, 0.00367, 0.00318, 0.0028, 0.00251, 0.00227
])
V = np.array([
    11.9, 11.82, 11.75, 11.59, 11.16, 10.89, 10.63, 10.51, 9.93, 9.71,
    9.1, 9.42, 8.95, 8.53, 8.14, 7.79, 7.46, 7.17, 6.65, 6.19,
    5.79, 5.44, 5.13, 4, 3.28, 2.78, 2.41, 2.12, 1.91, 1.72
])
sigma_V = np.full_like(V, 0.2)  # Error estándar proporcionado

# Definir modelos
def modelo_lineal(I, m, b):
    return m * I + b

def modelo_logaritmico(I, m, b):
    return m * np.log(I) + b

# Función de chi-cuadrado para cada modelo
def chi_squared(modelo, I, V, sigma_V, m, b):
    return np.sum(((V - modelo(I, m, b)) / sigma_V) ** 2)

# Simulación de Monte Carlo
n_simulaciones = 10000  # Número de simulaciones de Monte Carlo
m_min, m_max = -2000, 2000  # Rango de valores para m
b_min, b_max = -10, 20  # Rango de valores para b

# Inicialización de variables para almacenar el mínimo
chi2_min_lineal = np.inf
chi2_min_logaritmico = np.inf
best_m_lineal, best_b_lineal = 0, 0
best_m_logaritmico, best_b_logaritmico = 0, 0

for _ in range(n_simulaciones):
    # Generar valores aleatorios de m y b
    m_random = np.random.uniform(m_min, m_max)
    b_random = np.random.uniform(b_min, b_max)
    
    # Calcular chi-cuadrado para cada modelo
    chi2_lineal = chi_squared(modelo_lineal, I, V, sigma_V, m_random, b_random)
    chi2_logaritmico = chi_squared(modelo_logaritmico, I, V, sigma_V, m_random, b_random)
    
    # Verificar si es el mínimo encontrado hasta ahora para el modelo lineal
    if chi2_lineal < chi2_min_lineal:
        chi2_min_lineal = chi2_lineal
        best_m_lineal, best_b_lineal = 750, 0.00001

    # Verificar si es el mínimo encontrado hasta ahora para el modelo logarítmico
    if chi2_logaritmico < chi2_min_logaritmico:
        chi2_min_logaritmico = chi2_logaritmico
        best_m_logaritmico, best_b_logaritmico = m_random, b_random

# Mostrar los resultados
print(f"Modelo Lineal: Chi-cuadrado mínimo = {chi2_min_lineal:.4f} con m = {best_m_lineal:.4f}, b = {best_b_lineal:.4f}")
print(f"Modelo Logarítmico: Chi-cuadrado mínimo = {chi2_min_logaritmico:.4f} con m = {best_m_logaritmico:.4f}, b = {best_b_logaritmico:.4f}")

# Redefinir los rangos de m y b para la gráfica de contornos
m_values = np.linspace(m_min, m_max, 100)
b_values = np.linspace(b_min, b_max, 100)

# Crear una cuadrícula de m y b
M, B = np.meshgrid(m_values, b_values)

# Calcular chi-cuadrado para la cuadrícula
chi2_lineal = np.array([[chi_squared(modelo_lineal, I, V, sigma_V, m, b) for m in m_values] for b in b_values])
chi2_logaritmico = np.array([[chi_squared(modelo_logaritmico, I, V, sigma_V, m, b) for m in m_values] for b in b_values])

# Gráfica de contornos de chi-cuadrado
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.contour(M, B, chi2_lineal, levels=50, cmap='tab20c')
plt.plot(best_m_lineal, best_b_lineal, 'ro', label=f"Mínimo Monte Carlo (m={best_m_lineal:.2f}, b={best_b_lineal:.2f})")
plt.title(r'$\chi^2$ Contornos para Modelo Lineal ($V = mI + b$)')
plt.xlabel('Pendiente (m)')
plt.ylabel('Intersección (b)')
plt.colorbar(label=r'$\chi^2$')
plt.legend()

plt.subplot(1, 2, 2)
plt.contour(M, B, chi2_logaritmico, levels=20, cmap='plasma')
plt.plot(best_m_logaritmico, best_b_logaritmico, 'ro', label=f"Mínimo Monte Carlo (m={best_m_logaritmico:.2f}, b={best_b_logaritmico:.2f})")
plt.title(r'$\chi^2$ Contornos para Modelo Logarítmico ($V = m\log(I) + b$)')
plt.xlabel('Pendiente (m)')
plt.ylabel('Intersección (b)')
plt.colorbar(label=r'$\chi^2$')
plt.legend()

plt.tight_layout()
plt.show()

Modelo Lineal: Chi-cuadrado mínimo = 42.5459 con m = 750.0000, b = 0.0000
Modelo Logarítmico: Chi-cuadrado mínimo = 7875.2113 con m = 1.5265, b = 16.8708


KeyboardInterrupt: 